In [33]:
from datetime import datetime
import mt5
from MetaTrader5 import *
#from pytz import timezone
import matplotlib.pyplot as plt
#utc_tz = timezone('UTC')

# connect to MetaTrader 5
mt5.initialize()
# wait till MetaTrader 5 establishes connection to the 
# trade server and synchronizes the environment
#mt5.waitforterminal()

# request connection status and parameters
#print(mt5.TerminalInfo())
# get data on MetaTrader 5 version
#print(mt5.Version())
import pytz
import pandas as pd
import time
#pd.set_option('display.max_columns', 500) 
# number of columns to be displayed
#pd.set_option('display.width', 1500)      
# max table width to display

import ta
#import talib



currency_pair =["EURUSD","EURGBP","EURJPY","EURAUD","EURNZD",
"EURCHF","EURCAD","GBPUSD", "GBPJPY","GBPAUD","GBPNZD","GBPCAD",
"GBPCHF","USDCAD", "USDJPY","CHFJPY","CADJPY","NZDJPY","AUDJPY",
"AUDNZD","AUDCAD", "NZDUSD","NZDCAD","NZDCHF","AUDCHF","CADCHF",
"AUDUSD","USDCHF"]

time_frame=[mt5.TIMEFRAME_M1, mt5.TIMEFRAME_M2, mt5.TIMEFRAME_M3,
mt5.TIMEFRAME_M4, mt5.TIMEFRAME_M5, mt5.TIMEFRAME_M6, mt5.TIMEFRAME_M10,
mt5.TIMEFRAME_M12, mt5.TIMEFRAME_M15, mt5.TIMEFRAME_M20, mt5.TIMEFRAME_M30,
mt5.TIMEFRAME_H1, mt5.TIMEFRAME_H2, mt5.TIMEFRAME_H3,mt5.TIMEFRAME_H4,
mt5.TIMEFRAME_H6,mt5.TIMEFRAME_H8, mt5.TIMEFRAME_H12,mt5.TIMEFRAME_D1,
mt5.TIMEFRAME_W1,mt5.TIMEFRAME_MN1]

time_frame=["M1", "M2", "M3", "M4", "M5", "M6", "M10", "M12", 
"M15", "M20", "M30","H1", "H2", "H3", "H4", "H6", "H8", "H12", 
"D1", "W1", "MN1"]

# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")
now = datetime.now()
# create 'datetime' object in UTC time zone 
# to avoid the implementation of a local time zone offset
utc_from = datetime(2019, 4, 5, tzinfo=timezone)
 
lookback=50
# create DataFrame out of the obtained data
currency_strength = pd.DataFrame()
rsi_df=pd.DataFrame()

period=mt5.TIMEFRAME_H8
period=mt5.TIMEFRAME_H12
start=time.time()
for pair in currency_pair:
    ohlc = mt5.copy_rates_from(pair, period, datetime.now(), lookback)
    df = pd.DataFrame((ohlc),columns=['Time', 
    'Open', 'Low', 'High', 'Close', 'tickVolume', 
    'Spread', 'realVolume'])
    rsi_df[pair]=talib.RSI(df.Close,7)

rsi_df.tail()
strength=pd.DataFrame()
strength["USD"]=1/7*((100-rsi_df.EURUSD)+(100-rsi_df.GBPUSD)+\
    rsi_df.USDCAD+rsi_df.USDJPY+(100-rsi_df.NZDUSD)+\
        (100-rsi_df.AUDUSD)+rsi_df.USDCHF)
strength["EUR"]=1/7*(rsi_df.EURUSD+rsi_df.EURGBP+\
    rsi_df.EURJPY+rsi_df.EURAUD+rsi_df.EURNZD+rsi_df.EURCHF+\
        rsi_df.EURCAD)
strength["GBP"]=1/7*(rsi_df.GBPUSD+rsi_df.GBPJPY+\
    rsi_df.GBPAUD+rsi_df.GBPNZD+rsi_df.GBPCAD+rsi_df.GBPCHF+\
        (100-rsi_df.EURGBP))
strength["CHF"]=1/7*((100-rsi_df.EURCHF)+(100-rsi_df.GBPCHF)+\
    (100-rsi_df.NZDCHF)+(100-rsi_df.AUDCHF)+(100-rsi_df.CADCHF)+\
        rsi_df.CHFJPY+(100-rsi_df.USDCHF))
strength["JPY"]=1/7*((100-rsi_df.EURJPY)+(100-rsi_df.GBPJPY)+\
    (100-rsi_df.USDJPY)+(100-rsi_df.CHFJPY)+(100-rsi_df.CADJPY)+\
        (100-rsi_df.NZDJPY)+(100-rsi_df.AUDJPY))
strength["AUD"]=1/7*((100-rsi_df.EURAUD)+(100-rsi_df.GBPAUD)+\
    (100-rsi_df.AUDJPY)+rsi_df.AUDNZD+rsi_df.AUDCAD+
    rsi_df.AUDCHF+rsi_df.AUDUSD)
strength["CAD"]=1/7*((100-rsi_df.EURCAD)+(100-rsi_df.GBPCAD)+\
    (100-rsi_df.USDCAD)+rsi_df.CADJPY+(100-rsi_df.AUDCAD)+\
        (100-rsi_df.NZDCAD)+rsi_df.CADCHF)
strength["NZD"]=1/7*((100-rsi_df.EURNZD)+(100-rsi_df.GBPNZD)+\
    rsi_df.NZDJPY+rsi_df.NZDUSD+rsi_df.NZDCAD+rsi_df.NZDCHF+\
        (100-rsi_df.AUDNZD))

strength.tail()
strength.iloc[-2]
strength.iloc[-11]
end=time.time()
time_taken=end-start
print(time_taken/60)

plt.plot(strength.USD, label="USD")
plt.plot(strength.EUR, label="EUR")
plt.plot(strength.GBP, label="GBP")
plt.plot(strength.JPY, label="JPY")
plt.plot(strength.CAD, label="CAD")
plt.plot(strength.NZD, label="NZD")
plt.plot(strength.AUD, label="AUD")
plt.plot(strength.CHF, label="CHF")
plt.text(len(strength),strength.USD.iloc[-1], "USD")
plt.text(len(strength),strength.EUR.iloc[-1], "EUR")
plt.text(len(strength),strength.GBP.iloc[-1], "GBP")
plt.text(len(strength),strength.JPY.iloc[-1], "JPY")
plt.text(len(strength),strength.CAD.iloc[-1], "CAD")
plt.text(len(strength),strength.NZD.iloc[-1], "NZD")
plt.text(len(strength),strength.AUD.iloc[-1], "AUD")
plt.text(len(strength),strength.CHF.iloc[-1], "CHF")
plt.legend(loc="upper left")
plt.show()

#currency = ["USD","EUR","GBP","CHF","JPY","AUD","CAD","NZD"]


# shut down connection to MetaTrader 5
MT5Shutdown()
# request connection status and parameters
print(MT5TerminalInfo())

#function to plot strength
def plot_strength(strength):
    for x in currency:
        plt.plot(strength[x], label=x)
        plt.text(len(strength),strength[x].iloc[-1], x)
    plt.legend(loc="upper left")
    plt.show()

plot_strength(strength)


matching=[x for x in currency_pair if 'USD' in x]

def normalized_strength(strength):
    normalized_strength=pd.DataFrame()
    for z in currency:
        matching=[x for x in currency_pair if z in x]
        for x in matching:
            #normalized_strength[x]=strength[x[0:3]]-strength[x[3:6]]
            normalized_strength[x]=strength[x[0:3]]/strength[x[3:6]]
            normalized_strength[x]-=normalized_strength[x].mean()
            normalized_strength[x]/=normalized_strength[x].std()
    return normalized_strength

df=normalized_strength(strength)

AttributeError: module 'ta' has no attribute 'RSI'